In [8]:
import src.config as config

In [2]:
from datetime import datetime, timedelta

import pandas as pd

current_day = pd.to_datetime(datetime.utcnow()).floor('H')

#We fetch raw data for last 28 days, to add redudancy to our data pipeline
fetch_data_to = current_day
fetch_data_from = current_day - timedelta(days=28)
print(f'{fetch_data_from= }')
print(f'{fetch_data_to= }')

fetch_data_from= Timestamp('2024-08-11 03:00:00')
fetch_data_to= Timestamp('2024-09-08 03:00:00')


In [3]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}')

    # download 2 files from website
    rides = load_raw_data(year=from_date_.year, months=from_date_.month)
    rides = rides[rides.pickup_datetime >= from_date_]
    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]

    rides = pd.concat([rides, rides_2])

    # shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [4]:
rides = fetch_batch_raw_data(from_date= fetch_data_from, to_date= fetch_data_to)

from_date=Timestamp('2024-08-11 03:00:00'), to_date_=Timestamp('2023-09-10 03:00:00')
local file:  C:\Users\ingfz\ML Projects\taxi_demand_predictor\data\raw\rides_2023-08.parquet
File 2023-08 was already in local storage
local file:  C:\Users\ingfz\ML Projects\taxi_demand_predictor\data\raw\rides_2023-09.parquet
File 2023-09 was already in local storage


In [5]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 257/257 [00:00<00:00, 361.87it/s]


In [6]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_ts'],
    event_time='pickup_ts',
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/424087
Connected. Call `.close()` to terminate connection gracefully.


In [7]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 0.00% |          | Rows 0/172704 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/424087/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x23645b0a100>, None)